# Tutorial 2: Reduction and Normal Forms

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/buildLittleWorlds/types-normalization/blob/main/notebooks/02-reduction-and-normal-forms.ipynb)

> "Reduction is computation. A normal form is where computation stops." — Varen Tholl

## What Is Reduction?

In the passage calculus, computation proceeds by **reduction**: replacing a complex expression with a simpler one according to fixed rules.

The fundamental reduction is **beta-reduction**:

```
(λx.M) N  →  M[N/x]
```

Apply a function to an argument by substituting the argument for the parameter.

In [ ]:
import pandas as pd

BASE_URL = "https://raw.githubusercontent.com/buildLittleWorlds/densworld-datasets/main/data/"

# Load normalization traces to see reduction in action
traces_df = pd.read_csv(BASE_URL + "normalization_traces.csv")

# Look at a simple reduction sequence
traces_df[traces_df['trace_id'] == 'NT-001'][['trace_id', 'step_number', 'current_term', 'reduction_type']]

## Redexes: Where Reduction Happens

A **redex** (reducible expression) is a subterm that matches a reduction rule. For beta-reduction, a redex has the form `(λx.M) N`.

A term may contain multiple redexes. Which one should we reduce first?

In [ ]:
# A term with two redexes
# (λx.x) ((λy.y) z)
#
# Redex 1 (outer): (λx.x) applied to ((λy.y) z)
# Redex 2 (inner): (λy.y) applied to z

example = {
    'expression': '(λx.x) ((λy.y) z)',
    'redex_1': '(λx.x) ((λy.y) z)',
    'redex_1_position': 'outer',
    'redex_2': '(λy.y) z',
    'redex_2_position': 'inner'
}

print("Expression:", example['expression'])
print(f"Outer redex: {example['redex_1']}")
print(f"Inner redex: {example['redex_2']}")

## Reduction Strategies

A **reduction strategy** specifies which redex to reduce at each step. Varen Tholl catalogued several strategies (EV-935-005):

In [ ]:
# Load reduction strategies
strat_df = pd.read_csv(BASE_URL + "reduction_strategies.csv")

strat_df[['strategy_name', 'description', 'reduces_which', 'normalizing_behavior']].head(6)

### Call-by-Name vs. Call-by-Value

The two most common strategies differ in when they evaluate function arguments:

- **Call-by-name**: Reduce the outermost redex first. Arguments are substituted unevaluated.
- **Call-by-value**: Reduce arguments to values before substituting them.

In [ ]:
# Compare strategies on: (λx.x) ((λy.y) z)

call_by_name = [
    ('(λx.x) ((λy.y) z)', 'reduce outer'),
    ('(λy.y) z', 'reduce remaining'),
    ('z', 'normal form')
]

call_by_value = [
    ('(λx.x) ((λy.y) z)', 'reduce inner first'),
    ('(λx.x) z', 'now reduce outer'),
    ('z', 'normal form')
]

print("Call-by-Name:")
for term, action in call_by_name:
    print(f"  {term}  [{action}]")

print("\nCall-by-Value:")
for term, action in call_by_value:
    print(f"  {term}  [{action}]")

## Normal Forms

A term is in **normal form** if it contains no redexes — reduction is complete.

There are several flavors of normal form:

| Form | Definition | Example |
|------|------------|----------|
| Normal form | No redexes anywhere | `λx.x` |
| Head normal form | No redex at the head | `λx.(λy.y) x` |
| Weak head normal form | A lambda or application with variable head | `x M` |

In [ ]:
normal_forms = pd.DataFrame({
    'term': ['λx.x', 'λx.(λy.y) x', '(λx.x)(λy.y)', 'x y'],
    'normal_form': [True, False, False, True],
    'head_normal_form': [True, True, False, True],
    'weak_head_normal_form': [True, True, False, True],
    'has_redex': [False, True, True, False]
})

normal_forms

## Reduction Rules in Dependent Types

Linn's Calculus of Inductive Constructions has additional reduction rules beyond beta:

| Rule | Name | Description |
|------|------|-------------|
| β | Beta | `(λx:A.M) N → M[N/x]` |
| ι | Iota | Inductive eliminator applied to constructor |
| δ | Delta | Unfold a definition |
| ζ | Zeta | Let-binding reduction |
| η | Eta | `λx.(M x) → M` when x not in M |

In [ ]:
# Iota reduction: eliminator meets constructor
# rec_nat (succ n) base step  →  step n (rec_nat n base step)

iota_example = {
    'before': 'rec_nat (succ zero) base step',
    'after': 'step zero (rec_nat zero base step)',
    'after_again': 'step zero base',
    'rule': 'iota'
}

print("Iota reduction (structural recursion):")
print(f"  {iota_example['before']}")
print(f"  → {iota_example['after']}")
print(f"  → {iota_example['after_again']}")

## Why Reduction Strategies Matter

In an *untyped* calculus, different strategies can give different results:

- One strategy might terminate while another loops forever
- This is dangerous: the "answer" depends on how you compute

Tholl's key observation (EV-935-005):

> "Under strong normalization, all strategies terminate and reach the same normal form. Strategy becomes a matter of efficiency, not correctness."

In [ ]:
# The dangerous untyped case: (λx.y) Ω where Ω = (λx.x x)(λx.x x)
#
# Call-by-name: Reduces outer first → y (terminates)
# Call-by-value: Tries to evaluate Ω first → loops forever

danger = {
    'expression': '(λx.y) Ω',
    'call_by_name': 'y (terminates)',
    'call_by_value': 'diverges (tries to evaluate Ω)'
}

print(f"Expression: {danger['expression']}")
print(f"Call-by-name result: {danger['call_by_name']}")
print(f"Call-by-value result: {danger['call_by_value']}")
print("\nBut Ω is not typable, so this can't happen in typed calculus!")

## Visualizing Reduction Traces

In [ ]:
# Look at a more complex reduction
traces_df[traces_df['trace_id'] == 'NT-005'][['step_number', 'current_term', 'is_value']]

## Summary

| Concept | Definition |
|---------|------------|
| Redex | Subterm matching a reduction rule |
| Reduction | Replacing a redex with its contractum |
| Normal form | Term with no redexes |
| Strategy | Rule for choosing which redex to reduce |
| Strong normalization | All strategies terminate |

In the next tutorial, we'll see why different strategies reach the *same* normal form: confluence.

---

**Next Tutorial:** Confluence — The Church-Rosser property